# Projeto de Mineração de Dados (P2 e P3)

- Aluno: Luiz Fernando Costa dos Santos
- Matrícula: 20200025446


O dataset utilizado foi o Adult, que foi retirado do [UCI Machine Learning Repository](https://archive.ics.uci.edu/dataset/2/adult). Mais informações no arquivo `./data/adult.names` ou no próprio site mencionado acima.

## Importações

In [61]:
import pandas as pd 
import numpy as np
from typing import *
import hashlib
from sklearn.cluster import KMeans

## Pré-processamentos e carregamento dos dados

In [62]:
normalizacao = lambda x: x - x.min() / x.max() - x.min()

In [63]:
np.random.seed(40)

# ETL

In [64]:
class AdultETL:
    def __init__(self) -> None:
        self.features_names = [
            "age",
            "workclass",
            "fnlwgt",
            "education",
            "education-num",
            "marital-status",
            "occupation",
            "relationship",
            "race",
            "sex",
            "capital-gain",
            "capital-loss",
            "hours-per-week",
            "native-country",
            "income"
        ]
        
        self.initial_data_schema = data_types = {
            'age': 'int8',
            'workclass': 'object',
            'fnlwgt': 'int64',
            'education': 'object',
            'education-num': 'int8',
            'marital-status': 'category',
            'occupation': 'object',
            'relationship': 'category',
            'race': 'category',
            'sex': 'category',
            'capital-gain': 'int64',
            'capital-loss': 'int64',
            'hours-per-week': 'int8',
            'native-country': 'object'
        }
        self.cols_to_create_hash = [
            'age', 
            'workclass', 
            'fnlwgt', 
            'education', 
            'education-num', 
            'marital-status', 
            'occupation', 
            'relationship', 
            'sex', 
            'native-country'
        ]
        self.cols_to_cluster = [
            "age", 
            "fnlwgt", 
            "education-num", 
            "capital-gain", 
            "capital-loss", 
            "hours-per-week"
        ]
        
      
    def __call__(self) -> pd.DataFrame:
        return self.data
    
    def correct_data_types(self) -> None:
        for col, dtype in self.initial_data_schema.items():
            if col in self.data.columns:
                self.data[col] = self.data[col].astype(dtype)

    def create_people_id(self) -> None:
        concatenated_values = self.data[self.cols_to_create_hash].apply(lambda row: ''.join(map(str, row)), axis=1)
        hashed_values = concatenated_values.apply(lambda x: hashlib.sha256(x.encode()).hexdigest())
        self.data['people_id'] = hashed_values        
    
    def with_profit(self) -> None:
        self.data["capital-profit"] = self.data["capital-gain"] - self.data["capital-loss"]
    
    def with_categorized_data(self) -> None:
        string_cols = self.data.select_dtypes(include=["category"]).columns
        self.data[string_cols] = self.data[string_cols].apply(lambda x: x.str.strip()).astype("category")
        self.data["income"] = self.data["income"].apply(lambda x: 1 if x == " >50K" else 0).astype("category")    
        self.data["capital-profit-gt-0"] = self.data["capital-profit"].apply(lambda x: 1 if x > 0 else 0).astype("category")
    
    def with_kmeans(self, k: int, random_state: int = 42) -> None:
        kmeans = KMeans(n_clusters=k, random_state=random_state)
        
        input_data_norm = self.data[
            self.cols_to_cluster].apply(normalizacao)
        
        kmeans.fit(input_data_norm)
        self.data["cluster"] = kmeans.labels_
        self.data["cluster"] = self.data["cluster"].astype("category")
        
        
    def extract(self, data_name: str) -> None:
        self.data: pd.DataFrame = pd.read_csv(data_name, header=None)
        self.data.columns = self.features_names
        self.schema = self.data.dtypes
    
    def transform(self, k: int) -> None:
        self.correct_data_types()
        self.create_people_id()
        self.with_profit()
        self.with_categorized_data()
        self.with_kmeans(k)
        self.schema = self.data.dtypes
    
    def load(self, new_data_name: str, save_data_type = "parquet") -> None:
        if save_data_type == "parquet":
            self.data.to_parquet(new_data_name, index=False)
        elif save_data_type == "csv":
            self.data.to_csv(new_data_name, index=False)
        else:
            raise ValueError("Invalid save_data_type")
    

In [65]:
etl = AdultETL()

In [66]:
etl.extract("data/adult.data.csv")

In [67]:
etl.data.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [68]:
etl.schema

age                int64
workclass         object
fnlwgt             int64
education         object
education-num      int64
marital-status    object
occupation        object
relationship      object
race              object
sex               object
capital-gain       int64
capital-loss       int64
hours-per-week     int64
native-country    object
income            object
dtype: object

In [69]:
etl.transform(k=5)

In [70]:
etl.data.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income,people_id,capital-profit,capital-profit-gt-0,cluster
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,0,75429eb96e57d8aaa309fa099562db5761ca0c7bcc28c9...,2174,1,4
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,0,ac8849cd18724a2b1c9824a13399b022b9c7aeb414d23e...,0,0,4
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,0,e63a50cf2961ec987b435e549ad9795963dfb9d96ed3cb...,0,0,3
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,0,3fd2052cb4391f0a4c00c326864a5f2fd53e9eb032d9d2...,0,0,3
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,0,e5fb50b02464e84ca7b3a8e34ed2ed51c6898af5ec43f0...,0,0,1


In [71]:
etl.load("data/adult.data.processed.parquet")

In [72]:
etl.load("data/adult.data.processed.csv", save_data_type="csv")

In [73]:
etl.schema

age                        int8
workclass                object
fnlwgt                    int64
education                object
education-num              int8
marital-status         category
occupation               object
relationship           category
race                   category
sex                    category
capital-gain              int64
capital-loss              int64
hours-per-week             int8
native-country           object
income                 category
people_id                object
capital-profit            int64
capital-profit-gt-0    category
cluster                category
dtype: object

## Tabelas Dimensão e Fatos

In [74]:
parq = pd.read_parquet("data/adult.data.processed.parquet")

In [75]:
class FactsAndDimensions:
    def __init__(self, data: pd.DataFrame) -> None:
        self.data: pd.DataFrame = data
        self.dimensions: Dict[str, List[str, List[str]]] = {
            "country": [
                "country_id",
                ['native-country']
            ],
            "occupation": [
                "occupation_id",
                ['workclass', 'occupation', 'hours-per-week']
            ],    
            "capital": [
                "capital_id",
                ['capital-gain', 'capital-loss', 'income', 'cluster']
            ],
            "education": [
                "education_id",
                ['education', 'education-num']
            ],
            "marital": [
                'marital_id',
                ['marital-status', 'relationship']
            ],
            "basic_info": [
                'basic_info_id',
                ['age', 'race', 'sex']
            ]
        }
        self.facts: Dict[str, List[str]] = {
            "demographic": ['people_id', 'country_id', 'education_id', 'marital_id', 'basic_info_id'],
            "financial": ['occupation_id', 'capital_id', 'capital-profit', 'capital-profit-gt-0']
        }
        
        
    
    def __call__(self) -> Dict[str, pd.DataFrame]:
        return self.create_dimensions()
    
    @staticmethod   
    def create_id(data):
        concatenated_values = data.apply(lambda row: ''.join(map(str, row)), axis=1)
        return concatenated_values.apply(lambda x: hashlib.sha256(x.encode()).hexdigest())

    def with_ids(self) -> None:
        for _, (id_col, cols) in self.dimensions.items():
            self.data[id_col] = self.create_id(self.data[cols])
                
    def create_dimensions(self) -> None:
        self.dimensions_tables: Dict[str, pd.DataFrame] = {
            dimension: self.data[[id_col] + cols].drop_duplicates().reset_index(drop=True)
            for dimension, (id_col, cols) in self.dimensions.items()
        }
    
    def create_facts(self) -> None:
        self.facts_tables = {
            fact: self.data[cols]
            for fact, cols in self.facts.items()    
        }
        
    def save_all_tables(self) -> None:
        for name, table in self.dimensions_tables.items():
            table.to_parquet(f"data/dims/{name}.parquet", index=False)
        for name, table in self.facts_tables.items():
            table.to_parquet(f"data/facts/{name}.parquet", index=False)
        

In [76]:
fact_and_dims = FactsAndDimensions(parq)

In [77]:
fact_and_dims.with_ids()

In [78]:
fact_and_dims.create_dimensions()

In [79]:
fact_and_dims.create_facts()

In [80]:
fact_and_dims.save_all_tables()